# やること

## パラメータは？

パラメータとして与えるのは以下

①　$\rho_{\chi_{k}}$　:　 $\chi_{k}$に対するbeliefのprior precision。　candidateごとに用意する。

②　$\mu_{\chi_{k}}$　：　$\chi_{k}$に対するbeliefのmean。candidateごとに用意する。

③　$\rho_{\eta}$　：　シグナルのprecision。一つで良い。

④　$\chi_{k}$　：　candidate kのtrue quality。candidateごとに用意する。

## belief updating

③と④をgivenとして$s_{1,k}\sim N(\chi_{k}, \frac{1}{\rho_{\eta}})$をdrawする。

1期のシグナルをもとに、①②③を用いてposteriorを次のように構成する。

$B_{1, k} \sim N\left(\frac{\rho_{\chi_{k}} \mu_{\chi_{k}} + \rho_{\eta}s_{1, k}}{\rho_{\chi_{k}} + t \rho_{\eta}}, \frac{1}{\rho_{\chi_{k}} + t \rho_{\eta}}\right)$

このposterior meanを$E\left[\chi_{k}|\Omega_{1}\right]$として記録する。

以降t期においても同様に

③と④をgivenとして$s_{t,k}\sim N(\chi_{k}, \frac{1}{\rho_{\eta}})$をdrawする。

t期のシグナルをもとに、①②③を用いてposteriorを次のように構成する。

$B_{1, k} \sim N\left(\frac{\rho_{\chi_{k}} \mu_{\chi_{k}} + \rho_{\eta} \sum_{\tau = 1}^{t} s_{\tau, k}}{\rho_{\chi_{k}} + t \rho_{\eta}}, \frac{1}{\rho_{\chi_{k}} + t \rho_{\eta}}\right)$

そしてそのposterior meanを$E\left[\chi_{k}|\Omega_{t}\right]$として記録する。

## 詳細

パラメータセット一つにつきT回シグナルを乱数発生する。使うパラメータはいつも一緒なので出てくる数字に大差はない。

違うのはその出てくる順番。T!順あり得るので一つのパラメータセットにつきT!回1期からT期までの$E\left[\chi_{k}|\Omega_{t}\right]$を記録する。

さらにパラメータは100セット用意。

以下では、candidate4人について$E\left[\chi_{k}|\Omega_{t}\right]$のpathを100×T!通り作成するためのclassを作成する。

使用するクラスは以下で定義されている。


class ExpectedChi:
    
    def __init__(self, lam = 1.0, T = 5, sim = 10, random_state = 1):
        self.lam = lam # parameter of exponential distribution which rho's follow
        self.T = T # time
        self.sim = sim # simulation number
        self.random_state = random_state
    
    def setparam(self):
        # params[0, :]　：　rho_eta
        # params[(1,2,3,4), :] : rho_chi for each candidate
        # params[(5,6,7,8), :] : mu_chi for each candidate
        # params[(9,10,11,12), :] : chi for each candidate
        r = np.random.RandomState(self.random_state)
        rho = r.exponential(self.lam, (5, self.sim))
        mu_chi = r.normal(0.0,1.0,(8, self.sim))
        params = np.vstack((rho, mu_chi))
        return params
        
    def signals(self):
        # make the 4 dimentional array : all_signals
        # each element in all_signals is 3 dimentional array whose shape is (T! * 4 * T )
        params = self.setparam()
        all_signals = np.empty([self.sim, factorial(self.T).astype(int), 4, self.T])
        g = np.random.RandomState(self.random_state)
        for i in range(self.sim):
            for k in range(factorial(self.T).astype(int)):
                for j in range(4):
                    all_signals[i, k, j, :] = g.normal(params[j + 9, i], 1/params[0, i],  (1, self.T))
        return all_signals
    
    def cum_signals(self):
        # calculating cumulative signals for expected chi_k given omega
        sig = self.signals()
        for i in range(self.sim):
            for j in range(self.T):
                if j != 0:
                    sig[i, :, :, :][:, :, j] = sig[i, :, :, :][:, :, j] + sig[i, :, :, :][:, :, j -1]
        return sig
            
    def expected_chi(self):
        # calculating expected chi_k given omega
        cum = self.cum_signals()
        params = self.setparam()
        transform = np.empty((self.sim, factorial(self.T).astype(int), 4, self.T))
        
        # first multiply cum by rho_eta 
        for i in range(self.sim):
            transform[i, :, :, :] = cum[i, :, :, :] * params[0, i]
            
        # second add rho_chi + mu_chi to transform
        for i in range(self.sim):
            for j in range(4):
                transform[i, :, j, :] = transform[i, :, j, :] + (params[j + 1, i] * params[j + 5, i])
        
        # third divide by rho_chi + t * rho_eta
        for i in range(self.sim):
            for j in range(4):
                for t in range(self.T):
                    transform[i, :, j, t] = transform[i, :, j, t] / (params[j + 1, i] + t * params[0, i])
        
        return transform
    
    def means(self, each_sim = True):
        # calculate mean
        if each_sim == True:
            # calculate the means of expected chi for each parameter set and t
            # return matrix
            return self.expected_chi().mean(axis = (1,2))
        
        else:
            # calcualte the over all mean
            return self.expected_chi().mean(axis = (0,1,2))
        
    def variances(self, each_sim = True):
        # calculate variance
        if each_sim == True:
            # calculate the variances of expected chi for each parameter set and t
            # return list
            return self.expected_chi().var(axis = (1,2))
        
        else:
            # calcualte the over all variances
            return self.expected_chi().var(axis = (0,1,2))

In [1]:
import numpy as np
from scipy.special import factorial
from ExpectedChi import ExpectedChi as EC
test = EC()

In [2]:
test.means()

array([[-0.88396817, -0.20129887, -0.07476652, -0.01035578, -0.00456911],
       [ 0.11898259,  0.35318368,  0.37262632,  0.39299118,  0.38727952],
       [-0.1965428 , -0.14282227, -0.30184068, -0.20490944, -0.31136395],
       [-0.03435554, -0.07620991, -0.10602794, -0.13975592, -0.16922318],
       [ 0.92550285,  0.52809219,  0.31346203,  0.36058742,  0.37337207],
       [ 1.07895875,  0.84350256,  0.83603981,  0.80011803,  0.759201  ],
       [ 1.17690401,  0.94992759,  0.79176962,  0.72673968,  0.71392296],
       [ 4.04541776,  0.94819321,  0.79377798,  0.76571853,  0.74623071],
       [-4.35474123, -0.52890246, -0.47293046, -0.39411625, -0.34690467],
       [ 0.25674221,  0.3765129 ,  0.37933115,  0.39086378,  0.39165151]])

In [3]:
test.means(each_sim = False)

array([ 0.21329004,  0.30501786,  0.25314413,  0.26878812,  0.25395969])

In [4]:
test.variances()

array([[  2.60171109e+01,   1.67687640e+00,   1.04284883e+00,
          7.60145607e-01,   5.91060001e-01],
       [  2.54757633e+00,   1.03160013e+00,   7.75667226e-01,
          6.48415197e-01,   5.85140867e-01],
       [  9.84335859e+00,   1.63049119e+01,   2.32974304e+01,
          2.97891685e+01,   3.92828898e+01],
       [  9.82164151e-01,   1.03657862e+00,   1.02513588e+00,
          1.00557758e+00,   9.50730487e-01],
       [  3.11903977e+02,   1.97227377e+01,   9.43693874e+00,
          6.45701736e+00,   4.77318725e+00],
       [  3.07559084e+00,   4.09448538e+00,   4.55208968e+00,
          4.98334043e+00,   4.85648871e+00],
       [  3.38590786e+01,   8.34249230e+00,   4.76330570e+00,
          3.38546405e+00,   2.88221011e+00],
       [  2.21004143e+02,   4.99709725e+00,   2.55266236e+00,
          1.66578945e+00,   1.26164923e+00],
       [  8.80913642e+02,   7.73497059e+00,   4.10884829e+00,
          2.93607441e+00,   2.32785012e+00],
       [  1.77241750e+01,   1.7354251

In [5]:
test.variances(each_sim = False)

array([ 154.70521584,    6.91515931,    5.47019265,    5.41982897,
          5.99679726])